In [14]:
# import library

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
# 클래스 선언

# 입력 이미지 (3, 32, 32)
# 출력
classes = 10

class CIFAR10Net(nn.Module):
    def __init__(self):
        super(CIFAR10Net, self).__init__()
        # 합성곱 레이어 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 입력 채널: 3, 출력 채널: 32
        self.bn1 = nn.BatchNorm2d(32)  # 배치 정규화

        # 합성곱 레이어 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 입력 채널: 32, 출력 채널: 64
        self.bn2 = nn.BatchNorm2d(64)

        # 합성곱 레이어 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 입력 채널: 64, 출력 채널: 128
        self.bn3 = nn.BatchNorm2d(128)

        # 완전 연결 레이어
        self.fc1 = nn.Linear(128 * 4 * 4, 512)  # 128 채널의 4x4 특성 맵을 512 유닛으로 변환
        self.fc2 = nn.Linear(512, classes)  # 512 유닛을 10개의 클래스로 변환

    def forward(self, x):
        # 합성곱 레이어 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # 맥스 풀링

        # 합성곱 레이어 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # 맥스 풀링

        # 합성곱 레이어 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # 맥스 풀링

        # 완전 연결 레이어
        x = x.view(-1, 128 * 4 * 4)  # 텐서를 1차원으로 펼치기
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [36]:
# 메인함수

best_valid_loss = float('inf')
best_valid_acc = 0.0

def main():
    global best_valid_loss, best_valid_acc
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    lr = 0.001    # 학습률
    epochs = 2   # 실행횟수

    model = CIFAR10Net().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # 시각화를 위해 각 수치들을 저장하는 빈 리스트를 만들어준다.
    train_losses = []
    valid_losses = []
    train_accuracies = []
    valid_accuracies = []

    print("\033[34mtrain\033[0m")

    def train(model, device, train_loader, optimizer, epoch, epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        startStr = f'Epoch {epoch+1:2d}/{epochs} \033[34m' + 'Train ' + '\033[0m'
        with tqdm(total=len(train_loader), desc=startStr) as pbar:
            for data, target in train_loader:
                data, target = data.to(device), target.to(device)

                # 경사하강법 설정
                optimizer.zero_grad()
                outputs = model(data)
                loss = criterion(outputs, target)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * data.size(0)
                _, predicted = torch.max(outputs, 1)
                train_total += target.size(0)
                train_correct += (predicted == target).sum().item()
                pbar.update(1)
                
            train_loss = train_loss / train_total
            train_accuracy = 100 * train_correct / train_total
            train_losses.append(train_loss)
            train_accuracies.append(train_accuracy)

            str = f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.3f}%'
            pbar.set_postfix_str(str)

    def valid_or_test(mode, model, device, dataloader, num_epochs):
        model.eval()
        loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            if mode == 'valid':
                startStr = f'Epoch {epoch+1:2d}/{num_epochs} \033[34m' + 'Valid ' + '\033[0m'
            elif mode == 'test':
                startStr = f'Epoch {epoch+1:2d}/{num_epochs} \033[34m' + 'Test ' + '\033[0m'

            with tqdm(total=len(dataloader), desc=startStr) as pbar:
                for data, target in dataloader:
                    data, target = data.to(device), target.to(device)
                    outputs = model(data)
                    loss = criterion(outputs, target)

                    loss += loss.item() * data.size(0)
                    _, predicted = torch.max(outputs, 1)
                    total += target.size(0)
                    correct += (predicted == target).sum().item()
                    pbar.update(1)

                loss = loss / total
                accuracy = 100 * correct / total
                if mode == 'valid':
                    endStr = f'Valid Loss: {loss:.4f}, Valid Acc: {accuracy:.3f}%'
                elif mode == 'test':
                    endStr = f'Test Loss: {loss:.4f}, Test Acc: {accuracy:.3f}%'
                pbar.set_postfix_str(endStr)

        if mode == 'valid':
            valid_losses.append(loss)
            valid_accuracies.append(accuracy)
            print('-'*110)

    # 실행 (train, valid, test)
    for epoch in range(epochs):
        train(model, device, train_loader, optimizer, epoch, epochs)
        valid_or_test('valid', model, device, valid_loader, epoch)

        # 가중치 저장 조건 1
        # 가장 낮은 오차, 가장 높은 정확도
        if torch.tensor(valid_losses[-1]) < best_valid_loss:
            print("Saving best model based on loss...")
            best_valid_loss = valid_losses[-1]
            torch.save(model.state_dict(), 'best_model_loss.pt')
        elif valid_losses[-1] == best_valid_loss and valid_accuracies[-1] > best_valid_acc:
            best_valid_acc = valid_accuracies[-1]
            torch.save(model.state_dict(), 'best_model_loss.pt')


        # 가중치 저장 조건 2
        if valid_accuracies[-1] > best_valid_acc:
            best_valid_acc = valid_accuracies[-1]
            torch.save(model.state_dict(), 'best_model_acc.pt')
        elif valid_accuracies[-1] == best_valid_acc and torch.tensor(valid_losses[-1]).item() < best_valid_loss: 
            best_valid_loss = valid_losses[-1]
            torch.save(model.state_dict(), 'best_model_acc.pt')     


        # 모델 저장
        if torch.tensor(valid_losses[-1]).item() < best_valid_loss:
            print("Saving best model based on loss...")  # 디버깅용 print 문 추가
            best_valid_loss = valid_losses
            torch.save(model.state_dict(), 'best_model_loss.pt')

    valid_or_test('test', model, device, test_loader, test_loader)

    

In [37]:
# 메인함수 실행

if __name__ == '__main__':

    # 데이터 증식 & 정규화
    transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),   # 데이터 증강: 랜덤하게 잘라내기
    transforms.RandomHorizontalFlip(),      # 데이터 증강: 랜덤하게 좌우 반전
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),  # 정규화
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    # CIFAR10 데이터셋 불러오기
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

    # train, validation 비율 설정(8:2)
    train_ratio = 0.8

    train_size = int(len(train_dataset)*train_ratio)
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


    # 데이터 로더 생성
    batch_size = 128  # 원하는 배치 크기 설정
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)  # num_workers : cpu 개수
    valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    main()

Files already downloaded and verified
Files already downloaded and verified
train


Epoch  1/2 Train : 100%|█| 313/313 [00:14<00:00, 21.25it/s, Trai
Epoch  1/0 Valid : 100%|█| 79/79 [00:09<00:00,  8.15it/s, Valid 
C:\Users\한명수\AppData\Local\Temp\ipykernel_53108\863842222.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if torch.tensor(valid_losses[-1]) < best_valid_loss:
C:\Users\한명수\AppData\Local\Temp\ipykernel_53108\863842222.py:118: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if torch.tensor(valid_losses[-1]).item() < best_valid_loss:


--------------------------------------------------------------------------------------------------------------
Saving best model based on loss...


Epoch  2/2 Train : 100%|█| 313/313 [00:14<00:00, 21.48it/s, Trai
Epoch  2/1 Valid : 100%|█| 79/79 [00:10<00:00,  7.62it/s, Valid 


--------------------------------------------------------------------------------------------------------------
Saving best model based on loss...


Epoch  2/<torch.utils.data.dataloader.DataLoader object at 0x000
